In [ ]:
## functions for loading and transforming data

import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt

def fst(p): return (lambda (x,y) : x)(p)
def snd(p): return (lambda (x,y) : y)(p)
def last(ell): return ell[len(ell)-1]
def funEt(t,Et): return (t,snd(last(filter(lambda (x,_): x <= t, Et)))) ## dont use this...
def getMaxTime(Et): return fst(Et[len(Et)-1])
def getMaxTimeZZ((i,Et)): return fst(Et[len(Et)-1])
DENSITY = 0.01
PLOTFRACTION = 1 ## i.e. 1/x of all instances is plotted
HIGHLIGHT = 80 ## one in 10 simulations will be plotted in black
DENSITYrec = 1.0/DENSITY

def plottableEt(Et): 
    discretetimes = map(lambda x: DENSITY*x, range(0, int(DENSITYrec*getMaxTime(Et))))
    EtCopy = Et
    currentvalue = snd(EtCopy[0])
    EtCopy = EtCopy[1:]
    PEt = [] ## return this
    for t in discretetimes:
        if t < fst(EtCopy[0]):
            PEt += [(t,currentvalue)]
        else:
            while t >= fst(EtCopy[0]):
                currentvalue = snd(EtCopy[0])
                EtCopy = EtCopy[1:]
            PEt += [(t,currentvalue)]
    return PEt
    
def rearr(xs,ins): return [xs[i] for i in ins]

def loadAndPlot(filename,ax1,ax2,ax3,ax1a,ax2a,ax3a):
    file = open(filename)
    tree = ET.parse(file)
    root = tree.getroot()

    singleHostInstances = root.findall(".//*/within_host_history")
    singleHostInstances = [instance.findall("./within_host_statesummary") for instance in singleHostInstances]
    virss = [[statsum.find("./virus") for statsum in history] for history in singleHostInstances]

    timess = [[float(statsum.attrib["time"]) for statsum in history] for history in singleHostInstances]

    escapess = [[float(vir.attrib["e"]) for vir in virs] for virs in virss]
    reversionss = [[float(vir.attrib["f"]) + float(vir.attrib["d"])for vir in virs] for virs in virss]
    SpVLss = [[float(vir.attrib["SpVL"]) for vir in virs] for virs in virss]

    Ets = [zip(times,escapes) for (times,escapes) in zip(timess,escapess)]
    Fts = [zip(times,reversions) for (times,reversions) in zip(timess,reversionss)]
    Vts = [zip(times,SpVLs) for (times,SpVLs) in zip(timess,SpVLss)]
    
    ### sort Ets and remember the sorting, because it is the same for all

    sorting_indices = range(0,len(Ets))
    indexedEts = zip(sorting_indices,Ets)
    indexedEts = sorted(indexedEts,key=getMaxTimeZZ)
    indexedEts = indexedEts[::-1]
    Ets = map(snd,indexedEts)
    approxEts = map(plottableEt,Ets)
    sorting_indices = map(fst,indexedEts)

    Fts = rearr(Fts,sorting_indices)
    approxFts = map(plottableEt,Fts)

    Vts = rearr(Vts,sorting_indices)
    approxVts = map(plottableEt,Vts)


    ############### the number of mutations ###############

    meanEt = [[] for x in approxEts[0]]
    meanFt = [[] for x in approxFts[0]]

    tt = map(fst,approxEts[0])

    for Et in approxEts:
        xs = map(snd,Et)
        meanEt = [(lambda (a,b): a+[b])(p) for p in zip(meanEt,xs)] + meanEt[len(xs):]

    for Ft in approxFts:
        xs = map(snd,Ft)
        meanFt = [(lambda (a,b): a+[b])(p) for p in zip(meanFt,xs)] + meanFt[len(xs):]    

    meanEt = zip(tt,map(mean,meanEt))
    meanFt = zip(tt,map(mean,meanFt))

    ####################### virus load ######################

    meanVt = [[] for x in approxVts[0]]

    tt = map(fst,approxVts[0])

    for Vt in approxVts:
        xs = map(snd,Vt)
        meanVt = [(lambda (a,b): a+[b])(p) for p in zip(meanVt,xs)] + meanVt[len(xs):]
    
    meanVt = zip(tt,map(mean,meanVt))
    
    ax1.plot(map(fst,meanEt),map(snd,meanEt),color="purple",linewidth=2,zorder=2)
    ax1a.plot(map(fst,meanEt),map(snd,meanEt),color="purple",linewidth=2,zorder=2)
    ax2.plot(map(fst,meanFt),map(snd,meanFt),color="green",linewidth=2,zorder=2)
    ax2a.plot(map(fst,meanFt),map(snd,meanFt),color="green",linewidth=2,zorder=2)
    ax3.plot(map(fst,meanVt),map(snd,meanVt),color="blue",linewidth=2,zorder=2)
    ax3a.plot(map(fst,meanVt),map(snd,meanVt),color="blue",linewidth=2,zorder=2)

    for n,Et in enumerate(Ets):
        if n%PLOTFRACTION==0:
            color = 'black' if n%HIGHLIGHT==5 else 'violet'
            zorder = 2 if n%HIGHLIGHT==5 else 1
            linewidth = 1 if n%HIGHLIGHT==5 else 0.5
            Etstep = [ Et[i/2] if i%2==0 else (fst(Et[(i+1)/2]), snd(Et[(i-1)/2])) for i in range(2*len(Et)-1) ] + [last(Et)]
            ax1.plot(map(fst,Etstep), map(snd,Etstep),color=color,zorder=zorder,linewidth=linewidth)
            ax1a.plot(map(fst,Etstep), map(snd,Etstep),color=color,zorder=zorder,linewidth=linewidth)

    for n,Ft in enumerate(Fts):
        if n%PLOTFRACTION==0:
            color = 'black' if n%HIGHLIGHT==5 else 'lightgreen'
            zorder = 2 if n%HIGHLIGHT==5 else 1
            linewidth = 1 if n%HIGHLIGHT==5 else 0.5
            Ftstep = [ Ft[i/2] if i%2==0 else (fst(Ft[(i+1)/2]), snd(Ft[(i-1)/2])) for i in range(2*len(Ft)-1) ] + [last(Ft)]
            ax2.plot(map(fst,Ftstep), map(snd,Ftstep),color=color,zorder=zorder,linewidth=linewidth)
            ax2a.plot(map(fst,Ftstep), map(snd,Ftstep),color=color,zorder=zorder,linewidth=linewidth)
    endsEts = map(fst,filter(lambda x: snd(x)%PLOTFRACTION==0,zip(map(last,Ets),range(len(Ets)))))
    endsFts = map(fst,filter(lambda x: snd(x)%PLOTFRACTION==0,zip(map(last,Fts),range(len(Fts)))))
    ax1.scatter(map(fst,endsEts),map(snd,endsEts),1.3,color='purple',zorder=2)
    ax2.scatter(map(fst,endsFts),map(snd,endsFts),1.3,color='green',zorder=2)

    for n,Vt in enumerate(Vts):
        if n%PLOTFRACTION==0:
            color = 'black' if n%HIGHLIGHT==5 else 'lightblue'
            zorder = 2 if n%HIGHLIGHT==5 else 1
            linewidth = 1 if n%HIGHLIGHT==5 else 0.5
            Vtstep = [ Vt[i/2] if i%2==0 else (fst(Vt[(i+1)/2]), snd(Vt[(i-1)/2])) for i in range(2*len(Vt)-1) ] + [last(Vt)]
            ax3.plot(map(fst,Vtstep), map(snd,Vtstep),color=color,zorder=zorder,linewidth=linewidth)
            ax3a.plot(map(fst,Vtstep), map(snd,Vtstep),color=color,zorder=zorder,linewidth=linewidth)
    endsVts = map(fst,filter(lambda x: snd(x)%PLOTFRACTION==0,zip(map(last,Vts),range(len(Vts)))))
    ax3.scatter(map(fst,endsVts),map(snd,endsVts),1.3,color='blue',zorder=2)
    
    ax1.set_xlim(0,max(tt))
    D1 = 0.24;
    ax3a.set_xlim(0,D1)
    ax3a.set_xticks([0,D1])
    ax3a.set_xticklabels(['0','$D_1$'])
    maxt = max(tt)
    ax3.set_xlim(0,maxt)
    ax3.set_xticks(range(5,int(maxt),5))
    ax3.set_xlabel('time (years)')
    ax3.xaxis.set_label_coords(5./12., -0.2)

In [ ]:
%matplotlib inline
###### MAKE THE FIGURE

import matplotlib
import matplotlib.gridspec as gridspec
from numpy import mean
matplotlib.rcParams.update({'font.size': 18})

fig = plt.figure(figsize=[6, 7.5])
gs = gridspec.GridSpec(3, 2, width_ratios=[2, 12]) 
ax1 = fig.add_subplot(gs[1])
ax2 = fig.add_subplot(gs[3],sharex=ax1)
ax3 = fig.add_subplot(gs[5],sharex=ax1)
ax1a = fig.add_subplot(gs[0],sharey=ax1)
ax2a = fig.add_subplot(gs[2],sharex=ax1a,sharey=ax2)
ax3a = fig.add_subplot(gs[4],sharex=ax1a,sharey=ax3)

filenameA = 'data/singleHostsSims.xml'

loadAndPlot(filenameA,ax1,ax2,ax3,ax1a,ax2a,ax3a)

# Add a y axis label in a custom location

ax1a.set_ylabel('# escape\nmutations',horizontalalignment='center')
# ax1.set_xlabel('time (years)') ######### use shared x-axis
ylim_min, ylim_max = ax1.get_ylim()
ax1.set_ylim(max([0,ylim_min]),ylim_max)
ylim_min, ylim_max = ax2.get_ylim()
ax2.set_ylim(max([0,ylim_min]),ylim_max)
ax2a.set_ylabel('# deleterious\nmutations',horizontalalignment='center')

# Add a y axis label in a custom location

ax3a.set_ylabel('$\log_{10}$ virus load')
ylim_min, ylim_max = ax3.get_ylim()
ax3.set_ylim(max([0,ylim_min]),ylim_max)

plt.setp(ax1.get_xticklabels(), visible=False)
plt.setp(ax1a.get_xticklabels(), visible=False)
plt.setp(ax1.get_yticklabels(), visible=False)
plt.setp(ax2.get_xticklabels(), visible=False)
plt.setp(ax2a.get_xticklabels(), visible=False)
plt.setp(ax2.get_yticklabels(), visible=False)
plt.setp(ax3.get_yticklabels(), visible=False)

### align the y-labels
labelxpos = -1.1
ax1a.yaxis.set_label_coords(labelxpos, 0.5)
ax2a.yaxis.set_label_coords(labelxpos, 0.5)
ax3a.yaxis.set_label_coords(labelxpos+0.3, 0.5)

fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0.03, hspace=None)

fig.savefig('singlehost-plot.png')